In [80]:
import lxml.etree
import xml.etree.ElementTree as ET
import os 
import pandas as pd
from lxml import etree
parser = etree.XMLParser(recover=True)
filepath = 'DDI/4248_ddi_2.5.xml'


def get_tag_paths(root, path=""):
    """Recursively extracts all tag paths from an XML tree."""
    paths = []
    if path:
        paths.append(path)
    for child in root:
        paths.extend(get_tag_paths(child, f"{path}/{child.tag}"))
    return paths

tree = ET.parse(filepath)  # Replace with your XML file
root = tree.getroot()
paths = get_tag_paths(root)
for path in paths:
    print(path)


/{ddi:codebook:2_5}docDscr
/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation
/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}titlStmt
/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}titlStmt/{ddi:codebook:2_5}titl
/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}titlStmt/{ddi:codebook:2_5}IDNo
/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}prodStmt
/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}prodStmt/{ddi:codebook:2_5}producer
/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}prodStmt/{ddi:codebook:2_5}copyright
/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}prodStmt/{ddi:codebook:2_5}copyright/{ddi:codebook:2_5}ExtLink
/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}verStmt
/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}verStmt/{ddi:codebook:2_5}version
/{ddi:codeb

In [ ]:
files = os.listdir('DDI/')
xml_files = []

for file in files:
    if '.xml' in file:
        xml_files.append('DDI/' + file)

print(len(xml_files))
print(xml_files)

12
['DDI/100179_ddi_2.5.xml', 'DDI/100640_ddi_2.5.xml', 'DDI/101343_ddi_2.5.xml', 'DDI/106966_ddi_2.5.xml', 'DDI/20622_ddi_2.5.xml', 'DDI/36036_ddi_2.5.xml', 'DDI/36174_ddi_2.5.xml', 'DDI/37250_ddi_2.5.xml', 'DDI/38645_ddi_2.5.xml', 'DDI/38821_ddi_2.5.xml', 'DDI/39091_ddi_2.5.xml', 'DDI/4248_ddi_2.5.xml']


In [87]:
ds_title = './/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}titlStmt/{ddi:codebook:2_5}titl' # remove 'Metadata record for ' at beginning
ds_author = './/{ddi:codebook:2_5}stdyDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}rspStmt/{ddi:codebook:2_5}AuthEnty' # Affiliation is the attribute, no ORCiD
ds_doi = './/{ddi:codebook:2_5}stdyDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}titlStmt/{ddi:codebook:2_5}IDNo' # 2nd element is DOI URL
ds_keyword = './/{ddi:codebook:2_5}stdyDscr/{ddi:codebook:2_5}stdyInfo/{ddi:codebook:2_5}subject/{ddi:codebook:2_5}keyword' 
ds_abstract = './/{ddi:codebook:2_5}stdyDscr/{ddi:codebook:2_5}stdyInfo/{ddi:codebook:2_5}abstract'
ds_restriction = './/{ddi:codebook:2_5}stdyDscr/{ddi:codebook:2_5}dataAccs/{ddi:codebook:2_5}useStmt/{ddi:codebook:2_5}restrctn'
ds_access = './/{ddi:codebook:2_5}stdyDscr/{ddi:codebook:2_5}notes'
ds_date = './/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}verStmt/{ddi:codebook:2_5}version' # attribute is date
ds_funder = './/{ddi:codebook:2_5}stdyDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}prodStmt/{ddi:codebook:2_5}grantNo' # text is grant number, attribute is funder
ds_license_statement = './/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}prodStmt/{ddi:codebook:2_5}copyright'#in text
ds_license_url = './/{ddi:codebook:2_5}docDscr/{ddi:codebook:2_5}citation/{ddi:codebook:2_5}prodStmt/{ddi:codebook:2_5}copyright/{ddi:codebook:2_5}ExtLink' # URL in attribute

In [143]:
def separate_name(name):
    '''Separates a name into First, Middle (if applicable) and Last, returns these elements as separate strings.'''
    middle = ""
    
    # Check if the name contains a comma (indicating "last, first" or "last, first m." format)
    if ',' in name:
        parts = name.split(", ") # Split into a list: "last, first" becomes ["last", "first"]
        last = parts[0] # Last name is everything before the comma
        
        # Check if there's an element after the comma
        if len(parts) > 1:
            first_and_middle = parts[1].split() # Split whatever was after the comma with spaces
            first = first_and_middle[0] # First name will be the first part of that
            
            # Assign middle if available
            if len(first_and_middle) > 1: # If there is a second part
                middle = first_and_middle[1].replace(".", "") # Assign middle name as second part, remove . because DC will add it
        else:
            # Set first to an empty string if there's nothing after the comma
            first = ""
    else: # If the name had no comma, you can assume it is in First Last or First Middle Last format
        parts = name.split() # Split into a list where spaces are 
        
        first = parts[0] # First name is first element of list
        last = parts[-1] # Last name is the last element of the list 
        
        # Check if there is a middle name/initial 
        if len(parts) > 2: 
            middle = parts[1].replace(".", "") # Remove period if it exists
    
    return first, middle, last

def to_html(string):
    '''Takes in a string. If the string is not in html already (assume first character is <) wrap it in <p> ... </p>.'''
    if string[0] != "<":
        string = "<p>" + string + "</p>"
    return(string)

def list_to_string(lst):
    '''Takes in a list of strings ['a', 'b', 'c'] and returns a single string containing the list elements, separated by commas 'a, b, c'. '''
    if isinstance(lst, list):  # Check if the value is a list
        return ', '.join(lst)
    else:
        return ""  # Convert non-list values to string or handle as needed
    
def get_lists(root, field):
    elements = root.findall(field)
    text = []
    attributes = []
    for element in elements:
        text.append(element.text)
        attributes.append(element.attrib)
    return text, attributes

def add_titles(xml_files):
    titles = []
    for file in xml_files:
        tree = ET.parse(file, parser=parser)
        root = tree.getroot()
        title = get_lists(root, ds_title)[0][0]
        titles.append(to_html(title[20:])) # Remove 'Metadata record for '
    df = pd.DataFrame(titles, columns = ['title'])
    return df

def add_abstracts(xml_files, df):
    abstracts = []
    for file in xml_files:
        tree = ET.parse(file, parser=parser)
        root = tree.getroot()
        abstract = get_lists(root, ds_abstract)[0][0]
        abstracts.append(to_html(abstract))  
    df['abstract'] = abstracts
    return df

def add_date(xml_files, df):
    dates = []
    for file in xml_files:
        tree = ET.parse(file, parser=parser)
        root = tree.getroot()
        date = get_lists(root, ds_date)[1][0]['date']
        dates.append(date)  
    df['publication_date'] = dates
    return df

def add_keywords(xml_files, df):
    keywords = []
    for file in xml_files:
        tree = ET.parse(file, parser=parser)
        root = tree.getroot()
        keyword = get_lists(root, ds_keyword)[0]
        keywords.append(list_to_string(keyword))
    df['keywords'] = keywords
    return df

def add_doi(xml_files, df):
    dois = []
    for file in xml_files:
        tree = ET.parse(file, parser=parser)
        root = tree.getroot()
        doi = get_lists(root, ds_doi)[0][1]
        dois.append(doi)
    df['source_fulltext_url'] = dois
    return df	

In [144]:
# XML variables 


def build_dataframe(xml_files):
    df = add_titles(xml_files)
    df = add_abstracts(xml_files, df)
    df = add_date(xml_files, df)
    df = add_keywords(xml_files, df)
    df = add_doi(xml_files, df)

    return(df)



df_output = build_dataframe(xml_files)

display(df_output)


,title,abstract,publication_date,keywords,source_fulltext_url
0,<p>High Intensity Interval- vs Moderate Intens...,<p>Purpose: To compare the effects of six week...,2024-11-05,"Liproproteins, Blood, Obesity, Blood Pressure,...",http://doi.org/10.3886/E53793V1
1,<p>Randomization to Randomization Probability:...,"<p>This folder contains the data, code, and da...",2024-11-05,"caffeine, randomization method, vigilance, moo...",http://doi.org/10.3886/E100640V1
2,<p>A method for measuring human body compositi...,<p>The files required to replicate the results...,2024-11-05,"body fat, body volume, obesity , image analysis",http://doi.org/10.3886/E101343V1
3,<p>Illustration of measurement error models fo...,<p>The files\nrequired to reproduce the result...,2024-11-05,"measurement error, regression calibration, mul...",http://doi.org/10.3886/E106966V1
4,<p>Assessing Identity Theft Offenders' Strateg...,<p>The purpose of this study was to examine th...,2009-03-31,"access to information, adult offenders, federa...",http://doi.org/10.3886/ICPSR20622.v1
5,<p>Advanced Cognitive Training for Independent...,<p>The data producers have recompiled the ACTI...,2015-07-29,"aging population, cognitive functioning, cogni...",http://doi.org/10.3886/ICPSR36036.v1
6,<p>Randomized Controlled Trial of Breakfast Re...,"<p>This study is a multi-site, 3 parallel arm,...",2015-08-07,"diet, eating habits, health, health behavior, ...",http://doi.org/10.3886/ICPSR36174.v1
7,<p>Post Coital DNA Recovery in Minority Proxy ...,<p>Introduction and Background. Minorities are...,2019-12-17,"genetics data, reproductive history, sexually ...",http://doi.org/10.3886/ICPSR37250.v1
8,<p>2010 United States Census Tract Community T...,<p>This dataset contains two measures designed...,2023-03-07,"census data, census tract level, diabetes, epi...",http://doi.org/10.3886/ICPSR38645.v1
9,<p>Advanced Cognitive Training for Independent...,<p>ACTIVE (Advanced Cognitive Training for Ind...,2023-12-11,"aging population, cognitive functioning, elder...",http://doi.org/10.3886/ICPSR38821.v1
